In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba

**Exercise 7: Glacier growth**

Compute the evolution through time of a glacier assuming that it has zero thickness at $x=0$ and $x=L$ using an explicit finite difference scheme.

Follow the following steps:

From $s_i^k=b_i^k+h_i^k$, compute $A_i^k$

Compute $K_{i+1/2}^k$ using $h_{i+1/2}^k=\frac{h_i^k+h_{i+1}^k}{2}$ and $\partial_xs_{i+1/2}^k=\frac{s_{i+1}^k-s_i^k}{\Delta x}$

Compute the flux $q_{i+1/2}^k=K_{i+1/2}^k\partial_x s_{i+1/2}^k$

Compute the divergence of the flux $\partial_xq_i^k=\frac{q_{i+1/2}^k-q_{i-1/2}^k}{\Delta x}$

Find the optimal time step $\Delta t=0.125\frac{\Delta x^2}{\max K_{i+1/2}^k}$

Compute the new thickness $h_i^{k+1}=h_i^k+\Delta t(\partial_xq_i^k+A_i^k)$

In [ ]:
nx = 101

In [ ]:
rho=910
g=9.81
A=1.e-16
L=150.e3
A1=(2.*A*(rho*g)**3)/5


In [ ]:
x = np.linspace(0,L,nx)
b = 1500*np.exp(-(x-L/2)**2/(L/10)**2)
dx = L/(nx - 1)

dt0=1
ela=100
beta=0.07


In [ ]:
plt.plot(x,b)

In [ ]:
fig, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize=(10,6))

hp = np.ones(nx)
h = np.zeros(nx)
s = b.copy()
it = 0

tim=0
while np.max(np.abs(h-hp)) > 1e-3:
    it = it + 1
    hp = h.copy()
    am = beta*(s-ela)
    am = np.where(am>2, 2, am)
    hx = (h[0:nx-2] + h[1:nx-1])/2
    sn = np.abs(s[1:nx-1] - s[0:nx-2])/dx
    kd = A1*hx**5*sn**2
    q = kd*(s[1:nx-1] - s[0:nx-2])/dx
    dq = (q[1:nx-2] - q[0:nx-3])/dx
    dt = 0.125*(dx**2/np.max(kd))
    dt = np.where(dt>dt0, dt0, dt)
    tim = tim + dt
    h[1:nx-2] = h[1:nx-2] + dt*(dq+am[1:nx-2])
    h = np.where(h>0, h, 0)
    h[0] = 0
    h[-1] = 0
    s = b + h
    if it%1000==0:ax1.plot(x,s,'cyan',label=str(tim))

ax1.plot(x,b,'orange')
ax1.set_xlabel('$x$')
ax1.set_ylabel('$h$')
ax1.set_title('Growing glacier geometry');
ax1.legend()